# Initialization

In [1]:
## Import Libraries and Dependencies ##
import pandas as pd
import numpy as np

# Path

In [2]:
INPUT_PATH = "D:/OneDrive - National University of Singapore/NUS MTech KE/MTech KE - FYP - InsureSense/Kang Jiang/Phase 3/System Implementation/scripts/Data Mining & Machine Learning/dataset/sentiment_analysis/"

In [3]:
OUTPUT_PATH = "D:/OneDrive - National University of Singapore/NUS MTech KE/MTech KE - FYP - InsureSense/Kang Jiang/Phase 3/System Implementation/scripts/Data Mining & Machine Learning/dataset/user_location/"

# Import disaster tweets

In [4]:
# Start row & End row
STARTROW = 0
ENDROW = 20

In [5]:
# df_without_user_location = pd.read_csv(INPUT_PATH + "df_Japan_Floods_sentiment_labelled.csv", index_col=0, encoding = "ISO-8859-1")
df_without_user_location = pd.read_csv(INPUT_PATH + "df_Typhoon_Jebi_sentiment_labelled.csv", index_col=0)
# df_without_user_location = pd.read_csv(INPUT_PATH + "df_Typhoon_Mangkhut_sentiment_labelled.csv", index_col=0)

In [6]:
df_without_user_location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49727 entries, 0 to 49726
Data columns (total 14 columns):
tweet_id           49727 non-null int64
user               49727 non-null object
timestamp          49727 non-null object
date               49727 non-null object
events             49727 non-null object
text               49727 non-null object
processed_text     49727 non-null object
likes              49727 non-null int64
replies            49727 non-null int64
retweets           49727 non-null int64
url                49727 non-null object
disaster_flag      49727 non-null int64
category_phase     49727 non-null int64
sentiment_final    49727 non-null float64
dtypes: float64(1), int64(6), object(7)
memory usage: 5.7+ MB


## Extract tweet user registered location

In [7]:
## Import Libraries and Dependencies ##
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re

from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.exc import GeocoderTimedOut
URL_INIT = 'https://twitter.com/'

import time

In [8]:
df_user = df_without_user_location[['user']].drop_duplicates().reset_index(drop=True)
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23343 entries, 0 to 23342
Data columns (total 1 columns):
user    23343 non-null object
dtypes: object(1)
memory usage: 182.4+ KB


In [9]:
# Define the user range to extract
df_user = df_user[STARTROW:ENDROW]

In [10]:
# From all scraped tweets, get a set of unique usernames and put them in this list.
list_of_users = df_user['user'].tolist()
len(list_of_users)

20

In [11]:
# The located userlocations are appended to this list
list_of_found_userlocations = []

In [12]:
# The not located userlocations are appended to this list. 
# Maybe they contain some typo or something else like that. 
list_of_nonfound_userlocations = []

In [13]:
# function to remove "@" from user name
def parse_url(tweet_user):
    url = URL_INIT+ tweet_user.strip('@')
    return url

In [14]:
# Tweets user location scraping loop
for user in list_of_users:
    start_time = time.time()    
    try:
        url = parse_url(user)
        response = urlopen(url)
    except:
        continue
        
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    
    try:
        location = soup.find('span','ProfileHeaderCard-locationText').text.strip('\n').strip()
    except:
        location = ""
        
    if location:
        if ',' in location:
            splitted_location = location.split(',')
        else:
            splitted_location = re.split('|;|-|/|°|#', location)
        try:
            if splitted_location:
                located_location = geolocator.geocode(splitted_location[0], timeout=100)
            else:
                located_location = geolocator.geocode(location, timeout=100)
            if located_location:
                user_plus_location = (user, located_location)
                print(user_plus_location)
                list_of_found_userlocations.append(user_plus_location)
            else:
                user_plus_incorrect_location = (user, location)
                print(user_plus_incorrect_location)
                list_of_nonfound_userlocations.append(user_plus_incorrect_location)
        except GeocoderTimedOut as e:
            print("Error: geocode failed on input %s with message %s"%(location, e))
    elapsed_time = time.time() - start_time
    print(elapsed_time)

('@pedromj', Location(東京都, 関東地方, 日本, (34.6968642, 139.4049033, 0.0)))
4.167856454849243
('@breakingweather', Location(State College, Centre County, Pennsylvania, USA, (40.7944504, -77.8616386, 0.0)))
3.5734410285949707


C:\Users\purpl\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


('@Somnium410', Location(東京, 千代田区, 東京都, 関東地方, 100-0002, 日本, (35.6828387, 139.7594549, 0.0)))
3.6223132610321045
('@SaraFujimura', Location(Arizona, USA, (34.395342, -111.7632755, 0.0)))
3.7200510501861572
('@sasucce', 'rachel * 18 * they/she * USA')
3.531554698944092
('@bzwibs', Location(Earth, Lamb County, Texas, USA, (34.2331373, -102.4107493, 0.0)))
3.718057155609131
2.6648709774017334
('@kevinmeyerson', '長野県北佐久郡軽井沢町')
3.794851303100586
('@hepomodeler', Location(熊本県, 九州地方, 日本, (32.6450475, 130.6341345, 0.0)))
3.504626989364624
2.694791793823242
('@ZacharysWeather', Location(Arlington, Tarrant County, Texas, USA, (32.701939, -97.105624826688, 0.0)))
3.909546136856079
('@AMMotorworks', Location(St. Petersburg, Pinellas County, Florida, USA, (27.7703796, -82.6695085, 0.0)))
3.567457914352417
('@mcdosty', Location(Accra, Accra Metropolitan, Greater Accra Region, GA-183-8164, Ghana, (5.5600141, -0.2057437, 0.0)))
3.626303195953369
('@BunnyTokyoJP', Location(東京, 千代田区, 東京都, 関東地方, 100-0002,

In [15]:
# store the user with labelled lat long into a dataframe
index_list = list(range(len(list_of_found_userlocations)))
columns = ['user','location', 'latitude', 'longitude']

df_user_location = pd.DataFrame(index=index_list, columns=columns)

for index, item in enumerate(list_of_found_userlocations):

    try:
        df_user_location['user'].iloc[index] = item[0]
        df_user_location['location'].iloc[index] = item[1][0]
        df_user_location['latitude'].iloc[index] = item[1][1][0]
        df_user_location['longitude'].iloc[index] = item[1][1][1]
    except:
        continue

In [16]:
list_of_found_userlocations

[('@pedromj', Location(東京都, 関東地方, 日本, (34.6968642, 139.4049033, 0.0))),
 ('@breakingweather',
  Location(State College, Centre County, Pennsylvania, USA, (40.7944504, -77.8616386, 0.0))),
 ('@Somnium410',
  Location(東京, 千代田区, 東京都, 関東地方, 100-0002, 日本, (35.6828387, 139.7594549, 0.0))),
 ('@SaraFujimura', Location(Arizona, USA, (34.395342, -111.7632755, 0.0))),
 ('@bzwibs',
  Location(Earth, Lamb County, Texas, USA, (34.2331373, -102.4107493, 0.0))),
 ('@hepomodeler', Location(熊本県, 九州地方, 日本, (32.6450475, 130.6341345, 0.0))),
 ('@ZacharysWeather',
  Location(Arlington, Tarrant County, Texas, USA, (32.701939, -97.105624826688, 0.0))),
 ('@AMMotorworks',
  Location(St. Petersburg, Pinellas County, Florida, USA, (27.7703796, -82.6695085, 0.0))),
 ('@mcdosty',
  Location(Accra, Accra Metropolitan, Greater Accra Region, GA-183-8164, Ghana, (5.5600141, -0.2057437, 0.0))),
 ('@BunnyTokyoJP',
  Location(東京, 千代田区, 東京都, 関東地方, 100-0002, 日本, (35.6828387, 139.7594549, 0.0))),
 ('@UX_Japan',
  Location(

In [17]:
df_user_location.head(20)

,user,location,latitude,longitude
0,@pedromj,"東京都, 関東地方, 日本",34.6969,139.405
1,@breakingweather,"State College, Centre County, Pennsylvania, USA",40.7945,-77.8616
2,@Somnium410,"東京, 千代田区, 東京都, 関東地方, 100-0002, 日本",35.6828,139.759
3,@SaraFujimura,"Arizona, USA",34.3953,-111.763
4,@bzwibs,"Earth, Lamb County, Texas, USA",34.2331,-102.411
5,@hepomodeler,"熊本県, 九州地方, 日本",32.645,130.634
6,@ZacharysWeather,"Arlington, Tarrant County, Texas, USA",32.7019,-97.1056
7,@AMMotorworks,"St. Petersburg, Pinellas County, Florida, USA",27.7704,-82.6695
8,@mcdosty,"Accra, Accra Metropolitan, Greater Accra Regio...",5.56001,-0.205744
9,@BunnyTokyoJP,"東京, 千代田区, 東京都, 関東地方, 100-0002, 日本",35.6828,139.759


In [18]:
# Data Storage

In [19]:
df_user_location.to_csv(OUTPUT_PATH + "/df_Typhoon_Jebi_User_Location_" + str(STARTROW) + "_" + str(ENDROW) + ".csv", index=False)